<a href="https://colab.research.google.com/github/mureed-hussain-droty/CV_project/blob/main/ShinaMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving shina_eng_final.csv to shina_eng_final.csv


In [ ]:
!pip install -U transformers datasets sentencepiece --quiet
!pip install sacremoses
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
!pip install --upgrade transformers


In [ ]:
import warnings
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
)

# (optional) hide the FutureWarning about `tokenizer=` being deprecated
warnings.filterwarnings("ignore", category=FutureWarning)

# 1) Read your CSV into pandas and rename columns to `src`/`tgt`
df = pd.read_csv("shina_eng_final.csv", encoding="utf-8")
df = df.rename(columns={"Shina": "src", "English": "tgt"})

# 2) Convert to an HF Dataset and split
dataset = Dataset.from_pandas(df)
split   = dataset.train_test_split(test_size=0.1)
train_ds = split["train"]
eval_ds  = split["test"]

# 3) Load model & tokenizer
# model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 4) Tokenization function
max_length = 64

import torch
print(torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.get_device_name(0))

def tokenize_fn(examples):
    inputs  = tokenizer(
        examples["src"],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )
    targets = tokenizer(
        examples["tgt"],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# 5) Apply tokenization and drop raw columns
tokenized_train = train_ds.map(
    tokenize_fn,
    batched=True,
    remove_columns=["src", "tgt"],
)
tokenized_eval = eval_ds.map(
    tokenize_fn,
    batched=True,
    remove_columns=["src", "tgt"],
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./shina_translation_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=False,
    logging_dir="./logs",
    logging_steps=150,
    save_strategy="epoch",
    do_eval=True,         # ← enable evaluation
    eval_steps=0,         # ← skip in-step evals (only eval at epoch end)
    fp16=True,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none",
    remove_unused_columns=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 7) Kick off training
trainer.train()


True 0 Tesla T4


Map:   0%|          | 0/4398 [00:00<?, ? examples/s]

Map:   0%|          | 0/489 [00:00<?, ? examples/s]

Step,Training Loss
150,1.959400
300,0.289200
450,0.279600
600,0.245000
750,0.181600
900,0.172500
1050,0.176100
1200,0.121600
1350,0.102900
1500,0.105500


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=5500, training_loss=0.11837707254019651, metrics={'train_runtime': 4764.3931, 'train_samples_per_second': 9.231, 'train_steps_per_second': 1.154, 'total_flos': 5956897949614080.0, 'train_loss': 0.11837707254019651, 'epoch': 10.0})

In [ ]:
output_dir = "/content/drive/MyDrive/shina_translation_model"

# Save using trainer
trainer.save_model(output_dir)

# Optionally, re-save without safe serialization
model.save_pretrained(output_dir, safe_serialization=False)

# Save tokenizer
tokenizer.save_pretrained(output_dir)



('/content/drive/MyDrive/shina_translation_model/tokenizer_config.json',
 '/content/drive/MyDrive/shina_translation_model/special_tokens_map.json',
 '/content/drive/MyDrive/shina_translation_model/tokenizer.json')

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "/content/drive/MyDrive/shina_translation_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Test Shina → English
print("SH to EN:", translate("تھئ کلاسر کیاک شؙداریِےۡ ہن؟"))               # Expected: Why
print("SH to EN:", translate("تؙہ سکولجوۡ گوٹ گا توۡ جیک تھینو؟"))  # Expected: Today is very hot
print("SH to EN:", translate("مس گوۡٹئ کوم تھمؙس۔"))      # Expected: Where are you going?
print("SH to EN:", translate("نے یہ، مہ پُمُونّگِہ ایوار بُجمُس۔"))    # Expected: Father reads books
print("sh to en:", translate("تھئی جیک کوم ہن"))

# Test Eng To Shina
print("EN to SH:", translate("Why"))

SH to EN: How many children do you have in your class
SH to EN: What do you do at home after school
SH to EN: I do my homework
SH to EN: No, I am going there for the first time
sh to en: What is your job
EN to SH: Why
